In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings("ignore")

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.net1 = nn.Sequential(nn.Linear(2, 3))
        self.net2 = nn.Sequential(nn.Linear(3, 5))
        self.net3 = nn.Sequential(nn.Linear(5, 4))
        self.net4 = nn.Sequential(nn.Linear(4, 3))
        
    def forward(self, x):
        x = self.net1(x)
        x = F.softplus(x)
        x = self.net2(x)
        x = F.softplus(x)
        x = self.net3(x)
        x = F.softplus(x)
        x = self.net4(x)
        return x

In [3]:
model1 = torch.load('./model1.pkl')

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.net1 = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
        
        self.net2 = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
        
    def forward(self, x):
        y = self.net1(x)
        # z = self.net2(torch.cat([x, y.reshape(-1, 1)], dim=1))
        z = self.net2(x)
        return y, z

In [5]:
model2 = torch.load('./model2.pkl')

In [6]:
x=np.linspace(15,100,4251)
y=np.linspace(1200,2000,1451)

In [7]:
combinations = np.array(np.meshgrid(x, y)).T.reshape(-1,2)
combinations.shape

(6168201, 2)

In [8]:
data=torch.tensor(combinations).to (torch. float32)

In [9]:
temp1=model1(data)
temp1

tensor([[ 2.2493, 95.3813, 87.4350],
        [ 2.2504, 95.3840, 87.4347],
        [ 2.2516, 95.3867, 87.4343],
        ...,
        [ 8.0586, 99.5147, 74.7185],
        [ 8.0598, 99.5154, 74.7158],
        [ 8.0609, 99.5161, 74.7130]], grad_fn=<AddmmBackward0>)

In [10]:
temp2 = torch.cat((data,temp1), dim=1)
temp2

tensor([[  15.0000, 1200.0000,    2.2493,   95.3813,   87.4350],
        [  15.0000, 1200.5518,    2.2504,   95.3840,   87.4347],
        [  15.0000, 1201.1034,    2.2516,   95.3867,   87.4343],
        ...,
        [ 100.0000, 1998.8966,    8.0586,   99.5147,   74.7185],
        [ 100.0000, 1999.4482,    8.0598,   99.5154,   74.7158],
        [ 100.0000, 2000.0000,    8.0609,   99.5161,   74.7130]],
       grad_fn=<CatBackward0>)

In [11]:
temp3 = temp2[temp2[:, 2]<3]
temp3

tensor([[  15.0000, 1200.0000,    2.2493,   95.3813,   87.4350],
        [  15.0000, 1200.5518,    2.2504,   95.3840,   87.4347],
        [  15.0000, 1201.1034,    2.2516,   95.3867,   87.4343],
        ...,
        [  30.7400, 1200.0000,    2.9986,   96.4380,   86.3822],
        [  30.7400, 1200.5518,    2.9997,   96.4389,   86.3798],
        [  30.7600, 1200.0000,    2.9995,   96.4388,   86.3802]],
       grad_fn=<IndexBackward0>)

In [12]:
temp4 = temp3[temp3[:, 3]>=85]
temp4.shape,temp4

(torch.Size([257543, 5]),
 tensor([[  15.0000, 1200.0000,    2.2493,   95.3813,   87.4350],
         [  15.0000, 1200.5518,    2.2504,   95.3840,   87.4347],
         [  15.0000, 1201.1034,    2.2516,   95.3867,   87.4343],
         ...,
         [  30.7400, 1200.0000,    2.9986,   96.4380,   86.3822],
         [  30.7400, 1200.5518,    2.9997,   96.4389,   86.3798],
         [  30.7600, 1200.0000,    2.9995,   96.4388,   86.3802]],
        grad_fn=<IndexBackward0>))

In [13]:
ans = torch.cat(model2(temp4), dim=1)
ans,ans.shape

(tensor([[ 31.8981, 122.1471],
         [ 31.8900, 122.1653],
         [ 31.8820, 122.1834],
         ...,
         [ 25.9933,  47.4744],
         [ 25.9842,  47.4779],
         [ 25.9855,  47.4607]], grad_fn=<CatBackward0>),
 torch.Size([257543, 2]))

In [14]:
temp5 = torch.cat((temp4, ans), dim=1)
temp5, temp5.shape

(tensor([[  15.0000, 1200.0000,    2.2493,  ...,   87.4350,   31.8981,
           122.1471],
         [  15.0000, 1200.5518,    2.2504,  ...,   87.4347,   31.8900,
           122.1653],
         [  15.0000, 1201.1034,    2.2516,  ...,   87.4343,   31.8820,
           122.1834],
         ...,
         [  30.7400, 1200.0000,    2.9986,  ...,   86.3822,   25.9933,
            47.4744],
         [  30.7400, 1200.5518,    2.9997,  ...,   86.3798,   25.9842,
            47.4779],
         [  30.7600, 1200.0000,    2.9995,  ...,   86.3802,   25.9855,
            47.4607]], grad_fn=<CatBackward0>),
 torch.Size([257543, 7]))

In [15]:
temp6 = temp5[temp5[:, 6]>90]
temp6 = temp6[temp6[:, 6]<95]
temp6, temp6.shape

(tensor([[  19.0200, 1200.0000,    2.4378,  ...,   87.3019,   30.4668,
            94.8820],
         [  19.0200, 1200.5518,    2.4390,  ...,   87.3007,   30.4583,
            94.8985],
         [  19.0200, 1201.1034,    2.4401,  ...,   87.2995,   30.4498,
            94.9150],
         ...,
         [  21.0600, 1420.6897,    2.9987,  ...,   86.3819,   26.1197,
            90.1505],
         [  21.0600, 1421.2413,    2.9999,  ...,   86.3795,   26.1105,
            90.1902],
         [  21.0800, 1420.6897,    2.9997,  ...,   86.3800,   26.1118,
            90.0227]], grad_fn=<IndexBackward0>),
 torch.Size([15398, 7]))

In [16]:
temp7 = temp6[temp6[:, 5]<26.5]
temp7, temp7.shape

(tensor([[  20.2600, 1416.8276,    2.9520,  ...,   86.4753,   26.4973,
            94.9801],
         [  20.2800, 1416.2759,    2.9518,  ...,   86.4757,   26.4986,
            94.8125],
         [  20.2800, 1416.8276,    2.9529,  ...,   86.4734,   26.4895,
            94.8524],
         ...,
         [  21.0600, 1420.6897,    2.9987,  ...,   86.3819,   26.1197,
            90.1505],
         [  21.0600, 1421.2413,    2.9999,  ...,   86.3795,   26.1105,
            90.1902],
         [  21.0800, 1420.6897,    2.9997,  ...,   86.3800,   26.1118,
            90.0227]], grad_fn=<IndexBackward0>),
 torch.Size([1304, 7]))

In [17]:
temp8 = temp7[:, 6] - temp7[:, 5]
temp8.max()

tensor(68.8516, grad_fn=<MaxBackward1>)

In [18]:
temp9 = temp7[temp7[:, 6] - temp7[:, 5]>68.8]
temp9, temp9.shape

(tensor([[  20.4400, 1432.8276,    2.9945,   96.4344,   86.3904,   26.1610,
            94.9856],
         [  20.4600, 1433.9310,    2.9978,   96.4373,   86.3838,   26.1348,
            94.9375],
         [  20.4600, 1434.4828,    2.9990,   96.4383,   86.3814,   26.1256,
            94.9772]], grad_fn=<IndexBackward0>),
 torch.Size([3, 7]))

In [19]:
temp9[:, 6] - temp9[:, 5]

tensor([68.8246, 68.8027, 68.8516], grad_fn=<SubBackward0>)